In [86]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [87]:
from dynamixel import AX12, Arm
import time

In [90]:
PORT = '/dev/tty.usbmodem1101'
BAUD = 1000000

In [91]:
# Test connection to all servos by pinging
# We expect error=0 for all 6 servos.  See https://emanual.robotis.com/docs/en/dxl/protocol1/#error
with AX12(PORT, BAUD) as robot:
    robot.ping(0)
    robot.ping(1)
    robot.ping(2)
    robot.ping(3)
    robot.ping(4)
    robot.ping(5)
    # print('test next msg: ', robot.read_next_msg())
    print('test all msgs: ')
    for msg in robot.read_all_msgs():
        print(msg)

test all msgs: 
Status(id=0x00, error=00000000, data=0)
Status(id=0x01, error=00000000, data=0)
Status(id=0x02, error=00000000, data=0)
Status(id=0x03, error=00000000, data=0)
Status(id=0x04, error=00000000, data=0)
Status(id=0x05, error=00000000, data=0)


In [92]:
# Write speeds of all servos
with Arm(PORT, BAUD) as robot:
    robot.write_all(AX12.MOVING_SPEED, 25, nbytes=2)

In [17]:
# Command base servo to 0 degrees
with Arm(PORT, BAUD) as robot:
    robot.command_angle(0, 0)

In [93]:
# Disable (or enable) all servos
with Arm(PORT, BAUD) as robot:
    robot.write_all(AX12.TORQUE_ENABLE, 0)  # change to 1 to enable

In [94]:
# Command all servo angles
with Arm(PORT, BAUD) as robot:
    robot.command_angles_deg(0, 0, 0, 0, 0)

In [69]:
# Record all joint angles
def joint_angles_string(robot):
    joint_angles = robot.read_all_joint_angles_deg()
    return ' '.join([f'{angle:3.0f}' for angle in joint_angles])
with Arm(PORT, BAUD) as robot:
    while True:
        print(joint_angles_string(robot))
        break
        time.sleep(0.5)

 -3  65 -63 133  -3  73


In [ ]:
 -3  40 -38 120  -3  62
  -3  65 -63 133  -3  73

In [102]:
# Paint dipping trajectory
TRAJECTORY_QS_TOLS_TIMEOUTS_PAUSES = [
    ([0, 0, 0, 0, 0], None, None, 0),
    ([0, 50, 110, 0, 90], None, None, 0),
    # ([-8, 53, 130, 0, 61], 2, 3, 1),
    # ([-8, 53, 130, 5, 61], 2, 3, 1),
    # ([-8, 53, 130, -8, 61], 2, 3, 1),
    ([-3, 39, 120, -3, 62], 2, 3, 0),
    ([-3, 64, 133, -3, 73], 2, 3, 0),

    ([0, 50, 90, 0, 90], None, None, 0),
    ([0, 50, 60, 0, 90], None, None, 0),
    ([0, 0, 0, 0, 0], None, None, 0),
]

# Follow trajectory
with Arm(PORT, BAUD) as robot:
    robot.write_all(AX12.TORQUE_ENABLE, 1)
    for i, (q, tol, timeout, pause) in enumerate(TRAJECTORY_QS_TOLS_TIMEOUTS_PAUSES):
        print(f'Going to {q} ({i+1}/{len(TRAJECTORY_QS_TOLS_TIMEOUTS_PAUSES)})')
        # robot.go_to_blocking(q, cb=lambda: print(f'{joint_angles_string(robot):80}', end='\r'))
        print_debug = lambda: print(f'          {joint_angles_string(robot):80}', end='\r')
        if robot.go_to_blocking(q, tol=tol, timeout=timeout, cb=print_debug):
            print("SUCCESS!")
        else:
            print("\nTimeout...  Moving on")
        time.sleep(pause)

Going to [0, 0, 0, 0, 0] (1/7)
SUCCESS!
Going to [0, 50, 110, 0, 90] (2/7)
SUCCESS!   -0  51 -50 104  -0  89                                                         
Going to [-3, 39, 120, -3, 62] (3/7)
SUCCESS!   -3  39 -39 121  -3  63                                                         
Going to [-3, 64, 133, -3, 73] (4/7)
Lost data! 255 50 -49 132  -3  71                                                         
Lost data! 2
Lost data! 4
Lost data! 0
Lost data! 81
Lost data! 1
Lost data! 167
SUCCESS!   -3  61 -61 134  -3  72                                                         
Going to [0, 50, 90, 0, 90] (5/7)
Lost data! 255
Lost data! 3
Lost data! 4
Lost data! 0
Lost data! 176
Lost data! 3
Lost data! 69
SUCCESS!   -1  51 -50  95  -1  89                                                         
Going to [0, 50, 60, 0, 90] (6/7)
SUCCESS!   -1  51 -50  66  -1  89                                                         
Going to [0, 0, 0, 0, 0] (7/7)
SUCCESS!   -1  11            

In [101]:
# Painting on window position
REST = [0, 0, 0, 0, 0]
PAINT_ON_WINDOW = [0, 0, -25, 0, -60]
q = REST
# q = PAINT_ON_WINDOW

with Arm(PORT, BAUD) as robot:
    robot.write_all(AX12.TORQUE_ENABLE, 1)

    print_debug = lambda: print(f'          {joint_angles_string(robot):80}', end='\r')
    if robot.go_to_blocking(q, tol=tol, timeout=timeout, cb=print_debug):
        print("SUCCESS!")
    else:
        print("\nTimeout...  Moving on")

SUCCESS!   -1   0   0  -2  -1  -5                                                         
